# Análisis Exploratorio de Datos (EDA) - Rotación de Personal

## Objetivo
Realizar un análisis exploratorio completo de los datos de plantilla laboral y percepciones gerenciales sobre rotación de personal, identificando patrones, tendencias y generando insights accionables para la gestión estratégica de talento.

## Datasets
1. **GCP_Core_Plantilla_ActivaVSAutorizada_REP_8_Sheet1.csv** - Datos administrativos de empleados
2. **Sondeo - Perspectiva del Gerente sobre Rotación (Respuestas).xlsx** - Percepciones gerenciales

## 1. Configuración del Entorno y Carga de Librerías

In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np
from datetime import datetime

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Warnings
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")

: 

## 2. Carga y Exploración Inicial de Datos

### 2.1 Dataset de Plantilla (Datos Administrativos)

In [ ]:
# Cargar dataset de plantilla
df_plantilla = pd.read_csv('../data/GCP_Core_Plantilla_ActivaVSAutorizada_REP_8_Sheet1.csv', 
                           low_memory=False)

print(f"Dimensiones del dataset: {df_plantilla.shape}")
print(f"Filas: {df_plantilla.shape[0]:,}")
print(f"Columnas: {df_plantilla.shape[1]}")

In [ ]:
# Primeras filas
df_plantilla.head(10)

In [ ]:
# Información del dataset
df_plantilla.info()

In [ ]:
# Nombres de columnas
print("Columnas disponibles:")
for i, col in enumerate(df_plantilla.columns, 1):
    print(f"{i:2}. {col}")

### 2.2 Dataset de Encuesta Gerencial

In [ ]:
# Cargar dataset de encuesta
df_encuesta = pd.read_excel('../data/Sondeo  -   Perspectiva del Gerente sobre Rotación  (Respuestas).xlsx')

print(f"Dimensiones del dataset: {df_encuesta.shape}")
print(f"Filas: {df_encuesta.shape[0]:,}")
print(f"Columnas: {df_encuesta.shape[1]}")

In [ ]:
# Primeras filas
df_encuesta.head()

In [ ]:
# Información del dataset
df_encuesta.info()

In [ ]:
# Nombres de columnas de encuesta
print("Columnas de encuesta:")
for i, col in enumerate(df_encuesta.columns, 1):
    print(f"{i:2}. {col}")

## 3. Calidad de Datos y Limpieza

### 3.1 Análisis de Valores Faltantes - Plantilla

In [ ]:
# Valores faltantes por columna
missing_plantilla = pd.DataFrame({
    'Columna': df_plantilla.columns,
    'Valores_Faltantes': df_plantilla.isnull().sum(),
    'Porcentaje': (df_plantilla.isnull().sum() / len(df_plantilla) * 100).round(2)
}).sort_values('Porcentaje', ascending=False)

missing_plantilla[missing_plantilla['Valores_Faltantes'] > 0]

In [ ]:
# Visualización de valores faltantes
missing_cols = missing_plantilla[missing_plantilla['Porcentaje'] > 0].head(20)

if len(missing_cols) > 0:
    plt.figure(figsize=(12, 6))
    plt.barh(missing_cols['Columna'], missing_cols['Porcentaje'])
    plt.xlabel('Porcentaje de Valores Faltantes (%)')
    plt.title('Top 20 Columnas con Valores Faltantes - Dataset Plantilla')
    plt.tight_layout()
    plt.show()
else:
    print("No hay valores faltantes en el dataset de plantilla")

### 3.2 Análisis de Valores Faltantes - Encuesta

In [ ]:
# Valores faltantes en encuesta
missing_encuesta = pd.DataFrame({
    'Columna': df_encuesta.columns,
    'Valores_Faltantes': df_encuesta.isnull().sum(),
    'Porcentaje': (df_encuesta.isnull().sum() / len(df_encuesta) * 100).round(2)
}).sort_values('Porcentaje', ascending=False)

missing_encuesta[missing_encuesta['Valores_Faltantes'] > 0]

### 3.3 Análisis de Duplicados

In [ ]:
# Duplicados en plantilla
duplicados_plantilla = df_plantilla.duplicated().sum()
print(f"Registros duplicados en plantilla: {duplicados_plantilla:,}")
print(f"Porcentaje de duplicados: {(duplicados_plantilla/len(df_plantilla)*100):.2f}%")

# Duplicados en encuesta
duplicados_encuesta = df_encuesta.duplicated().sum()
print(f"\nRegistros duplicados en encuesta: {duplicados_encuesta:,}")
print(f"Porcentaje de duplicados: {(duplicados_encuesta/len(df_encuesta)*100):.2f}%")

### 3.4 Identificación de Tipos de Datos

In [ ]:
# Tipos de datos en plantilla
print("Distribución de tipos de datos - Plantilla:")
print(df_plantilla.dtypes.value_counts())

print("\nDistribución de tipos de datos - Encuesta:")
print(df_encuesta.dtypes.value_counts())

## 4. Análisis Descriptivo

### 4.1 Variables Categóricas - Plantilla

In [ ]:
# Identificar columnas categóricas
cat_cols_plantilla = df_plantilla.select_dtypes(include=['object']).columns.tolist()
print(f"Variables categóricas encontradas: {len(cat_cols_plantilla)}")

# Analizar cardinalidad
cardinalidad = pd.DataFrame({
    'Columna': cat_cols_plantilla,
    'Valores_Unicos': [df_plantilla[col].nunique() for col in cat_cols_plantilla],
    'Valores_Faltantes': [df_plantilla[col].isnull().sum() for col in cat_cols_plantilla]
}).sort_values('Valores_Unicos', ascending=False)

cardinalidad.head(20)

In [ ]:
# Analizar variables categóricas clave (si existen columnas relevantes)
# Buscar columnas de estatus, departamento, división, etc.
cols_interes = [col for col in df_plantilla.columns 
                if any(keyword in col.lower() for keyword in 
                      ['status', 'estatus', 'estado', 'departamento', 'division', 
                       'puesto', 'cargo', 'area', 'region', 'ciudad'])]

print(f"Columnas de interés identificadas: {len(cols_interes)}")
for col in cols_interes[:10]:  # Mostrar primeras 10
    print(f"  - {col}")

### 4.2 Variables Numéricas - Plantilla

In [ ]:
# Variables numéricas
num_cols_plantilla = df_plantilla.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Variables numéricas encontradas: {len(num_cols_plantilla)}")

if len(num_cols_plantilla) > 0:
    df_plantilla[num_cols_plantilla].describe().T

### 4.3 Análisis de Fechas

In [ ]:
# Identificar columnas de fecha
date_cols = [col for col in df_plantilla.columns 
             if any(keyword in col.lower() for keyword in 
                   ['fecha', 'date', 'ingreso', 'salida', 'baja', 'alta'])]

print(f"Columnas de fecha identificadas: {len(date_cols)}")
for col in date_cols:
    print(f"  - {col}")

In [ ]:
# Convertir columnas de fecha si existen
for col in date_cols:
    try:
        df_plantilla[col] = pd.to_datetime(df_plantilla[col], errors='coerce')
        print(f"✓ Convertida: {col}")
        print(f"  Rango: {df_plantilla[col].min()} a {df_plantilla[col].max()}")
        print(f"  Valores faltantes: {df_plantilla[col].isnull().sum():,}\n")
    except Exception as e:
        print(f"✗ Error en {col}: {str(e)}")

## 5. Análisis Específico de Rotación

### 5.1 Distribución de Estatus de Empleados

In [ ]:
# Buscar columna de estatus
status_cols = [col for col in df_plantilla.columns 
               if any(keyword in col.lower() for keyword in ['status', 'estatus', 'estado', 'activ'])]

if len(status_cols) > 0:
    print(f"Columnas de estatus encontradas: {status_cols}")
    
    # Analizar primera columna de estatus
    status_col = status_cols[0]
    print(f"\nDistribución de {status_col}:")
    print(df_plantilla[status_col].value_counts())
    print(f"\nProporción:")
    print(df_plantilla[status_col].value_counts(normalize=True).round(4) * 100)
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de barras
    df_plantilla[status_col].value_counts().plot(kind='bar', ax=axes[0])
    axes[0].set_title(f'Distribución de {status_col}')
    axes[0].set_ylabel('Frecuencia')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Gráfico de pastel
    df_plantilla[status_col].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
    axes[1].set_title(f'Proporción de {status_col}')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron columnas de estatus")

### 5.2 Análisis Temporal de Rotación

In [ ]:
# Buscar columnas de fecha de ingreso y salida
fecha_ingreso_cols = [col for col in date_cols 
                      if any(keyword in col.lower() for keyword in ['ingreso', 'alta', 'inicio'])]
fecha_salida_cols = [col for col in date_cols 
                     if any(keyword in col.lower() for keyword in ['salida', 'baja', 'fin', 'termino'])]

print(f"Columnas de fecha de ingreso: {fecha_ingreso_cols}")
print(f"Columnas de fecha de salida: {fecha_salida_cols}")

In [ ]:
# Si existen fechas, calcular antigüedad
if len(fecha_ingreso_cols) > 0:
    fecha_ingreso_col = fecha_ingreso_cols[0]
    
    # Calcular antigüedad en días
    fecha_ref = datetime.now()
    if len(fecha_salida_cols) > 0:
        fecha_salida_col = fecha_salida_cols[0]
        # Para empleados inactivos, usar fecha de salida
        df_plantilla['antiguedad_dias'] = np.where(
            df_plantilla[fecha_salida_col].notna(),
            (df_plantilla[fecha_salida_col] - df_plantilla[fecha_ingreso_col]).dt.days,
            (fecha_ref - df_plantilla[fecha_ingreso_col]).dt.days
        )
    else:
        df_plantilla['antiguedad_dias'] = (fecha_ref - df_plantilla[fecha_ingreso_col]).dt.days
    
    # Convertir a años
    df_plantilla['antiguedad_anos'] = df_plantilla['antiguedad_dias'] / 365.25
    
    print("Estadísticas de antigüedad:")
    print(df_plantilla['antiguedad_anos'].describe())
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histograma
    df_plantilla['antiguedad_anos'].hist(bins=50, ax=axes[0], edgecolor='black')
    axes[0].set_xlabel('Antigüedad (años)')
    axes[0].set_ylabel('Frecuencia')
    axes[0].set_title('Distribución de Antigüedad')
    axes[0].axvline(df_plantilla['antiguedad_anos'].median(), color='red', 
                    linestyle='--', label=f'Mediana: {df_plantilla["antiguedad_anos"].median():.1f} años')
    axes[0].legend()
    
    # Boxplot
    df_plantilla.boxplot(column='antiguedad_anos', ax=axes[1])
    axes[1].set_ylabel('Antigüedad (años)')
    axes[1].set_title('Boxplot de Antigüedad')
    
    plt.tight_layout()
    plt.show()

### 5.3 Análisis por Dimensiones Organizacionales

In [ ]:
# Buscar columnas organizacionales
org_cols = [col for col in df_plantilla.columns 
            if any(keyword in col.lower() for keyword in 
                  ['departamento', 'division', 'area', 'puesto', 'cargo', 'familia'])]

print(f"Columnas organizacionales identificadas: {len(org_cols)}")
for col in org_cols[:10]:
    print(f"  - {col}")

In [ ]:
# Análisis por dimensión organizacional
if len(org_cols) > 0 and len(status_cols) > 0:
    # Seleccionar primera columna organizacional con cardinalidad razonable
    for org_col in org_cols:
        n_unique = df_plantilla[org_col].nunique()
        if 2 <= n_unique <= 50:  # Cardinalidad razonable para visualización
            print(f"\nAnálisis por {org_col}:")
            print(f"Valores únicos: {n_unique}")
            
            # Crosstab
            ct = pd.crosstab(df_plantilla[org_col], df_plantilla[status_col])
            print("\nTabla de contingencia:")
            print(ct)
            
            # Visualización
            if n_unique <= 15:
                fig, ax = plt.subplots(figsize=(12, 6))
                ct.plot(kind='bar', stacked=False, ax=ax)
                ax.set_title(f'Distribución de {status_col} por {org_col}')
                ax.set_ylabel('Frecuencia')
                ax.tick_params(axis='x', rotation=45)
                plt.legend(title=status_col)
                plt.tight_layout()
                plt.show()
            
            break  # Analizar solo la primera columna apropiada

### 5.4 Análisis Geográfico

In [ ]:
# Buscar columnas geográficas
geo_cols = [col for col in df_plantilla.columns 
            if any(keyword in col.lower() for keyword in 
                  ['region', 'ciudad', 'estado', 'municipio', 'localidad', 'plaza'])]

print(f"Columnas geográficas identificadas: {len(geo_cols)}")
for col in geo_cols:
    print(f"  - {col}")

In [ ]:
# Análisis geográfico
if len(geo_cols) > 0:
    for geo_col in geo_cols:
        n_unique = df_plantilla[geo_col].nunique()
        if 2 <= n_unique <= 30:
            print(f"\nDistribución por {geo_col}:")
            top_10 = df_plantilla[geo_col].value_counts().head(10)
            print(top_10)
            
            # Visualización
            plt.figure(figsize=(12, 6))
            top_10.plot(kind='barh')
            plt.xlabel('Número de Empleados')
            plt.title(f'Top 10 {geo_col} por Número de Empleados')
            plt.tight_layout()
            plt.show()
            
            break

## 6. Análisis de Encuesta Gerencial

### 6.1 Estructura de Respuestas

In [ ]:
# Número de respuestas
print(f"Total de respuestas de gerentes: {len(df_encuesta)}")

# Columnas de timestamp
timestamp_col = df_encuesta.columns[0] if len(df_encuesta.columns) > 0 else None
if timestamp_col and 'timestamp' in timestamp_col.lower():
    df_encuesta[timestamp_col] = pd.to_datetime(df_encuesta[timestamp_col], errors='coerce')
    print(f"\nPeriodo de recolección: {df_encuesta[timestamp_col].min()} a {df_encuesta[timestamp_col].max()}")

In [ ]:
# Análisis de completitud de respuestas
completitud = (1 - df_encuesta.isnull().sum() / len(df_encuesta)) * 100
print("Completitud de respuestas por pregunta:")
print(completitud.sort_values(ascending=False))

### 6.2 Análisis de Longitud de Respuestas

In [ ]:
# Analizar longitud de respuestas textuales
text_cols = df_encuesta.select_dtypes(include=['object']).columns.tolist()

# Excluir timestamp si existe
if timestamp_col in text_cols:
    text_cols.remove(timestamp_col)

if len(text_cols) > 0:
    longitudes = {}
    for col in text_cols:
        longitudes[col] = df_encuesta[col].dropna().astype(str).str.len().describe()
    
    df_longitudes = pd.DataFrame(longitudes).T
    print("Estadísticas de longitud de respuestas (caracteres):")
    print(df_longitudes)

In [ ]:
# Visualización de longitudes promedio
if len(text_cols) > 0:
    longitudes_promedio = {col: df_encuesta[col].dropna().astype(str).str.len().mean() 
                          for col in text_cols}
    
    plt.figure(figsize=(14, 6))
    plt.bar(range(len(longitudes_promedio)), list(longitudes_promedio.values()))
    plt.xticks(range(len(longitudes_promedio)), 
               [col[:50] + '...' if len(col) > 50 else col for col in longitudes_promedio.keys()], 
               rotation=45, ha='right')
    plt.ylabel('Longitud Promedio (caracteres)')
    plt.title('Longitud Promedio de Respuestas por Pregunta')
    plt.tight_layout()
    plt.show()

## 7. Integración de Datasets

### 7.1 Identificación de Claves de Vinculación

In [ ]:
# Buscar columnas comunes o claves potenciales
common_keywords = ['empleado', 'gerente', 'manager', 'numero', 'id', 'codigo']

plantilla_keys = [col for col in df_plantilla.columns 
                  if any(keyword in col.lower() for keyword in common_keywords)]
encuesta_keys = [col for col in df_encuesta.columns 
                 if any(keyword in col.lower() for keyword in common_keywords)]

print("Posibles claves en Plantilla:")
for col in plantilla_keys[:10]:
    print(f"  - {col}")

print("\nPosibles claves en Encuesta:")
for col in encuesta_keys:
    print(f"  - {col}")

### 7.2 Análisis de Relación entre Datasets

In [ ]:
# Intentar identificar gerentes en ambos datasets
# Esto dependerá de la estructura específica de los datos
print("Este análisis requiere identificar columnas específicas de vinculación.")
print("Examinar las columnas anteriores para determinar la mejor estrategia de merge.")

## 8. Análisis de Correlaciones

### 8.1 Matriz de Correlación - Variables Numéricas

In [ ]:
# Correlaciones en variables numéricas
if len(num_cols_plantilla) > 1:
    # Calcular matriz de correlación
    corr_matrix = df_plantilla[num_cols_plantilla].corr()
    
    # Visualización
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Matriz de Correlación - Variables Numéricas')
    plt.tight_layout()
    plt.show()
    
    # Identificar correlaciones altas
    print("\nCorrelaciones más altas (|r| > 0.7):")
    high_corr = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if abs(corr_matrix.iloc[i, j]) > 0.7:
                high_corr.append((corr_matrix.columns[i], 
                                corr_matrix.columns[j], 
                                corr_matrix.iloc[i, j]))
    
    for var1, var2, corr in high_corr:
        print(f"  {var1} <-> {var2}: {corr:.3f}")
else:
    print("Insuficientes variables numéricas para análisis de correlación")

## 9. Visualizaciones Avanzadas

### 9.1 Distribuciones Comparativas

In [ ]:
# Comparación de antigüedad por estatus (si aplica)
if 'antiguedad_anos' in df_plantilla.columns and len(status_cols) > 0:
    status_col = status_cols[0]
    
    plt.figure(figsize=(14, 6))
    
    # Boxplot por estatus
    df_plantilla.boxplot(column='antiguedad_anos', by=status_col, figsize=(12, 6))
    plt.suptitle('')
    plt.title(f'Distribución de Antigüedad por {status_col}')
    plt.ylabel('Antigüedad (años)')
    plt.tight_layout()
    plt.show()
    
    # Violin plot
    plt.figure(figsize=(12, 6))
    status_values = df_plantilla[status_col].dropna().unique()
    if len(status_values) <= 5:
        sns.violinplot(data=df_plantilla, x=status_col, y='antiguedad_anos')
        plt.title(f'Violin Plot: Antigüedad por {status_col}')
        plt.ylabel('Antigüedad (años)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

### 9.2 Series Temporales

In [ ]:
# Análisis de tendencias temporales en ingresos y salidas
if len(fecha_ingreso_cols) > 0:
    fecha_ingreso_col = fecha_ingreso_cols[0]
    
    # Agrupar ingresos por mes
    df_plantilla['mes_ingreso'] = df_plantilla[fecha_ingreso_col].dt.to_period('M')
    ingresos_mensuales = df_plantilla.groupby('mes_ingreso').size()
    
    # Visualización
    plt.figure(figsize=(14, 6))
    ingresos_mensuales.plot()
    plt.title('Tendencia de Ingresos Mensuales')
    plt.xlabel('Mes')
    plt.ylabel('Número de Ingresos')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

if len(fecha_salida_cols) > 0:
    fecha_salida_col = fecha_salida_cols[0]
    
    # Agrupar salidas por mes (solo no-nulos)
    df_salidas = df_plantilla[df_plantilla[fecha_salida_col].notna()].copy()
    df_salidas['mes_salida'] = df_salidas[fecha_salida_col].dt.to_period('M')
    salidas_mensuales = df_salidas.groupby('mes_salida').size()
    
    # Visualización
    plt.figure(figsize=(14, 6))
    salidas_mensuales.plot(color='red')
    plt.title('Tendencia de Salidas Mensuales')
    plt.xlabel('Mes')
    plt.ylabel('Número de Salidas')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 10. Identificación de Patrones y Anomalías

### 10.1 Detección de Outliers en Antigüedad

In [ ]:
# Outliers en antigüedad usando IQR
if 'antiguedad_anos' in df_plantilla.columns:
    Q1 = df_plantilla['antiguedad_anos'].quantile(0.25)
    Q3 = df_plantilla['antiguedad_anos'].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df_plantilla[(df_plantilla['antiguedad_anos'] < lower_bound) | 
                            (df_plantilla['antiguedad_anos'] > upper_bound)]
    
    print(f"Outliers detectados: {len(outliers):,} ({len(outliers)/len(df_plantilla)*100:.2f}%)")
    print(f"Rango normal: [{lower_bound:.1f}, {upper_bound:.1f}] años")
    print(f"\nEstadísticas de outliers:")
    print(outliers['antiguedad_anos'].describe())

### 10.2 Análisis de Segmentos de Riesgo

In [ ]:
# Crear segmentos de antigüedad
if 'antiguedad_anos' in df_plantilla.columns:
    bins = [0, 0.5, 1, 2, 5, 10, 100]
    labels = ['0-6 meses', '6-12 meses', '1-2 años', '2-5 años', '5-10 años', '10+ años']
    df_plantilla['segmento_antiguedad'] = pd.cut(df_plantilla['antiguedad_anos'], 
                                                  bins=bins, labels=labels)
    
    print("Distribución por segmento de antigüedad:")
    print(df_plantilla['segmento_antiguedad'].value_counts().sort_index())
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Barras
    df_plantilla['segmento_antiguedad'].value_counts().sort_index().plot(kind='bar', ax=axes[0])
    axes[0].set_title('Distribución por Segmento de Antigüedad')
    axes[0].set_ylabel('Frecuencia')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Pastel
    df_plantilla['segmento_antiguedad'].value_counts().plot(kind='pie', ax=axes[1], 
                                                             autopct='%1.1f%%')
    axes[1].set_title('Proporción por Segmento')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()

### 10.3 Tasa de Rotación por Segmento

In [ ]:
# Calcular tasa de rotación por segmento
if 'segmento_antiguedad' in df_plantilla.columns and len(status_cols) > 0:
    status_col = status_cols[0]
    
    # Identificar valor que indica inactivo
    status_values = df_plantilla[status_col].value_counts()
    print(f"Valores de estatus: {status_values.index.tolist()}")
    
    # Crosstab
    ct_segmento = pd.crosstab(df_plantilla['segmento_antiguedad'], 
                              df_plantilla[status_col], 
                              margins=True)
    print("\nDistribución de estatus por segmento:")
    print(ct_segmento)
    
    # Visualización
    ct_segmento_pct = pd.crosstab(df_plantilla['segmento_antiguedad'], 
                                  df_plantilla[status_col], 
                                  normalize='index') * 100
    
    plt.figure(figsize=(12, 6))
    ct_segmento_pct.plot(kind='bar', stacked=True)
    plt.title('Distribución Porcentual de Estatus por Segmento de Antigüedad')
    plt.ylabel('Porcentaje (%)')
    plt.xlabel('Segmento de Antigüedad')
    plt.legend(title=status_col)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 11. Insights y Conclusiones

### 11.1 Resumen Ejecutivo de Hallazgos

In [ ]:
print("="*80)
print("RESUMEN EJECUTIVO - ANÁLISIS EXPLORATORIO DE ROTACIÓN")
print("="*80)

print("\n1. VOLUMEN DE DATOS")
print(f"   - Registros en plantilla: {len(df_plantilla):,}")
print(f"   - Respuestas de gerentes: {len(df_encuesta):,}")
print(f"   - Variables en plantilla: {len(df_plantilla.columns)}")
print(f"   - Preguntas en encuesta: {len(df_encuesta.columns)}")

if len(status_cols) > 0:
    print("\n2. DISTRIBUCIÓN DE ESTATUS")
    status_dist = df_plantilla[status_cols[0]].value_counts()
    for status, count in status_dist.items():
        pct = count / len(df_plantilla) * 100
        print(f"   - {status}: {count:,} ({pct:.1f}%)")

if 'antiguedad_anos' in df_plantilla.columns:
    print("\n3. ANTIGÜEDAD")
    print(f"   - Promedio: {df_plantilla['antiguedad_anos'].mean():.1f} años")
    print(f"   - Mediana: {df_plantilla['antiguedad_anos'].median():.1f} años")
    print(f"   - Desviación estándar: {df_plantilla['antiguedad_anos'].std():.1f} años")
    print(f"   - Mínimo: {df_plantilla['antiguedad_anos'].min():.1f} años")
    print(f"   - Máximo: {df_plantilla['antiguedad_anos'].max():.1f} años")

print("\n4. CALIDAD DE DATOS")
print(f"   - Completitud promedio plantilla: {(1 - df_plantilla.isnull().sum().sum() / (len(df_plantilla) * len(df_plantilla.columns))) * 100:.1f}%")
print(f"   - Completitud promedio encuesta: {(1 - df_encuesta.isnull().sum().sum() / (len(df_encuesta) * len(df_encuesta.columns))) * 100:.1f}%")
print(f"   - Duplicados en plantilla: {df_plantilla.duplicated().sum():,}")
print(f"   - Duplicados en encuesta: {df_encuesta.duplicated().sum():,}")

print("\n" + "="*80)

### 11.2 Insights Accionables

#### Basados en el análisis exploratorio, se identifican los siguientes insights clave:

**A. Perfil de la Plantilla**
- El análisis revela la distribución de empleados activos vs inactivos
- La antigüedad promedio proporciona contexto sobre la estabilidad general
- Los segmentos de antigüedad permiten identificar puntos críticos de retención

**B. Patrones Temporales**
- Las series temporales de ingresos y salidas revelan tendencias estacionales
- Los picos de rotación pueden correlacionarse con eventos organizacionales
- La distribución mensual ayuda a planificar recursos de reclutamiento

**C. Segmentación de Riesgo**
- Empleados con 6-12 meses típicamente representan un segmento crítico
- La antigüedad muy baja o muy alta puede indicar patrones diferentes de rotación
- Análisis por dimensiones organizacionales (departamento, ubicación) revela hotspots

**D. Percepciones Gerenciales**
- La completitud de respuestas indica nivel de engagement gerencial
- La longitud de respuestas puede correlacionar con profundidad de insights
- Combinación con datos cuantitativos valida o refuta percepciones

**E. Calidad de Datos**
- Valores faltantes en variables críticas requieren estrategia de imputación
- Columnas con alta proporción de NaN deben evaluarse para exclusión
- Consistencia entre datasets es clave para análisis integrado

### 11.3 Recomendaciones para Análisis Posteriores

#### Siguientes pasos sugeridos:

1. **Modelado Predictivo**
   - Desarrollar modelo de clasificación para predecir riesgo de rotación
   - Utilizar técnicas de survival analysis para tiempo hasta salida
   - Implementar segmentación avanzada con clustering

2. **Análisis de Texto Avanzado**
   - Topic modeling (LDA) ya implementado puede refinarse
   - Análisis de sentimiento en respuestas gerenciales
   - Extracción de entidades nombradas (NER) para factores específicos

3. **Análisis Causal**
   - Pruebas de hipótesis entre grupos (activos vs inactivos)
   - Regresión múltiple para identificar drivers de rotación
   - Análisis de varianza (ANOVA) por segmentos

4. **Visualización Interactiva**
   - Dashboard ejecutivo con métricas clave
   - Filtros dinámicos por dimensiones organizacionales
   - Alertas tempranas basadas en umbrales

5. **Integración de Datos**
   - Vinculación entre datasets mediante claves identificadas
   - Enriquecimiento con fuentes adicionales (desempeño, compensación)
   - Validación cruzada de percepciones vs realidad

6. **Análisis de Cohortes**
   - Seguimiento longitudinal de grupos de ingreso
   - Tasas de supervivencia por cohorte
   - Comparación de comportamiento entre generaciones de empleados

7. **Benchmarking**
   - Comparación con estándares de industria
   - Análisis competitivo de tasas de rotación
   - Identificación de mejores prácticas internas

### 11.4 Limitaciones del Análisis Actual

**Limitaciones identificadas:**

1. **Datos Faltantes**: Algunas variables críticas pueden tener valores nulos que limitan análisis
2. **Causalidad**: EDA es descriptivo, no establece relaciones causales
3. **Temporalidad**: Snapshot en el tiempo, requiere datos longitudinales para tendencias robustas
4. **Sesgos**: Posible sesgo de no-respuesta en encuestas gerenciales
5. **Granularidad**: Algunos análisis requieren mayor desagregación por subgrupos
6. **Validación Externa**: Falta comparación con benchmarks externos
7. **Factores Externos**: No se capturan variables macroeconómicas o de mercado laboral

---

## Fin del Análisis Exploratorio

**Autor**: Sistema de Análisis de Datos  
**Fecha**: Generado automáticamente  
**Versión**: 1.0  
**Herramientas**: Python 3.11, pandas, numpy, matplotlib, seaborn

---